# Przetwarzanie Grafiki i Muzyki - laboratorium nr 4

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Zadanie 1 (1.5 punkta)
Proszę zaimplementować (ręcznie) konwersję z formatu RGB na YCbCr (https://en.wikipedia.org/wiki/YCbCr) i przetestować na wybranym przez siebie obrazku. Wzór:

$$
\begin{align}
Y &= 0.299 \cdot R + 0.587\cdot G + 0.114\cdot B \\
C_B &= 0.5 -0.169\cdot R - 0.331 \cdot G + 0.5\cdot B \\
C_R &= 0.5 + 0.5\cdot R - 0.419\cdot G - 0.081\cdot B,
\end{align}
$$

gdzie $R, G, B \in [0,1]$.

In [3]:
def bgr_to_ycbcr(image):
    new_image = np.copy(image).astype(np.float32)
    new_image = new_image / 255
    
    xform = np.array([[.299, .587, .114], [-.169, -.331, .5], [.5, -.419, -.081]])
    new_image = new_image.dot(xform.T)
    new_image[:,:,[1, 2]] += 0.5

    return new_image.astype(np.float32)

eagle = cv2.imread("./img/eagle.jpg", cv2.IMREAD_UNCHANGED)
cv2.imshow("original", eagle)

converted_to_YCbCr = bgr_to_ycbcr(eagle)

cv2.imshow("YCbCr", converted_to_YCbCr)

ycbcr_eagle = cv2.cvtColor(eagle, cv2.COLOR_BGR2YCrCb)
cv2.imshow("YCbCr_cv2", ycbcr_eagle)

cv2.waitKey(50)
cv2.destroyAllWindows()


## Zadanie 2 (1.5 punkta)

Proszę zaimplementować ręcznie konwersję z formatu YCbCr na RGB (https://en.wikipedia.org/wiki/YCbCr). Przetestuj kod na obrazie wynikowym z poprzedniego zadania. Wzór:

$$
\begin{align}
    R &= Y + 1.402*(C_R - 0.5) \\
    G &= Y - 0.344*(C_B - 0.5)-0.714*(C_R - 0.5) \\
    B &= Y + 1.772*(C_B - 0.5)
\end{align}
$$

gdzie $Y, C_R, C_B \in [0, 1]$.

In [9]:
def ycbcr_to_bgr(image):
    new_image = np.copy(image).astype(np.float32)
    new_image = new_image / 255
    
    R = new_image[:, :, 0] + 1.402 * (new_image[:, :, 1] - 0.5)
    G = new_image[:, :, 0] - 0.344 * (new_image[:, :, 1] - 0.5) - 0.714 * (new_image[:, :, 2] - 0.5)
    B = new_image[:, :, 0] + 1.772 * (new_image[:, :, 1] - 0.5)
    
    new_image = np.dstack((B, G, R))
    new_image = new_image * 255
    return new_image.astype(np.int8)
    
eagle = cv2.imread("./img/eagle.jpg", cv2.IMREAD_UNCHANGED)
cv2.imshow("original", eagle)

converted_to_YCbCr = bgr_to_ycbcr(eagle)
cv2.imshow("to_ycbcr", converted_to_YCbCr)

back_to_bgr = ycbcr_to_bgr(converted_to_YCbCr)
cv2.imshow("ycbcr_to_bgr", back_to_bgr)

cv2.waitKey(50)
cv2.destroyAllWindows()

## Zadanie 3 (1 punkt)
Proszę zaimplementować konwersję z formatu RGB na HLS i wyświetlić składowe H, L oraz S. Tutaj można użyć gotowej metody z pakietu OpenCV (cv2.COLOR_BGR2HLS).

In [52]:
eagle = cv2.imread("./img/eagle.jpg", cv2.IMREAD_COLOR)
hls_eagle = cv2.cvtColor(eagle, cv2.COLOR_BGR2HLS)
hls_separated = np.concatenate((hls_eagle[:, :, 0], hls_eagle[:, :, 1], hls_eagle[:, :, 2]), axis=1)
# or 
# hls_separated = np.hstack((hls_eagle[:, :, 0], hls_eagle[:, :, 1], hls_eagle[:, :, 2]))

cv2.imshow("eagle", eagle)
cv2.imshow("hls_eagle", hls_eagle)
cv2.imshow("hls_separated", hls_separated)

cv2.waitKey(50)
cv2.destroyAllWindows()

## Zadanie 4 (2 punkty)

Proszę stworzyć obrazek zawierający tylko zielone papryczki (analogicznie do przykładu z wykładu). Użyj obrazku "peppers.png".

HINT - zielone papryczki znajdują się tam, gdzie $R\in [20, 120]$ i $G \in [20, 160]$. R i G to natężenie składowych koloru w modelu RGB.

In [ ]:
def detect_green_peppers(image):
    image_with_peppers = np.copy(image)
    mask = np.all((image[:,:,1] >= 20, image[:,:,1] <= 160, image[:,:,2] >= 20, image[:,:,2] <= 120), axis=0)
    image_with_peppers[~mask] = 0

    return image_with_peppers

peppers = cv2.imread("./img/peppers.png", cv2.IMREAD_UNCHANGED)
cv2.imshow("peppers", peppers)
peppers_only = detect_green_peppers(peppers)
# without_peppers = detect_green_peppers(peppers)
# peppers_only = peppers - without_peppers
# 
cv2.imshow("peppers_only", peppers_only)
# cv2.imshow("without_peppers", without_peppers)
# cv2.imshow("peppers_only", peppers_only)
# 
cv2.waitKey(50)
cv2.destroyAllWindows()

## Zadanie 5 (2 punkty)
Proszę dokonać detekcji skóry na zdjęciu "face.jpg", używając modelu HLS.

HINT - skóra znajduje się tam, gdzie spełniona jest następująca reguła:
$$
\left(S \geq 50\right) \land \left(0.5 < \frac{L}{S} < 3 \right) \land \left( \left(H \leq 14\right) \lor \left(H \geq 165\right) \right) 
$$

![alt](https://raw.githubusercontent.com/przem85/PGiM/master/img/face.jpg)

In [5]:
def detect_face(image):
    new_image = cv2.cvtColor(image, cv2.COLOR_BGR2HLS)
    H, L, S = cv2.split(new_image)
    skin_mask = (S >= 50) & (0.5 < L / S) & (L / S < 3) & ((H <= 14) | (H >= 165))
    
    # why no work? :(
    # new_image[~skin_mask] = [0, 0, 0]
    # return new_image
    
    skin_detection = np.zeros_like(face_img)
    skin_detection[skin_mask] = image[skin_mask]
    return skin_detection
    
face_img = cv2.imread("./img/face.jpg", cv2.IMREAD_COLOR)
cv2.imshow("original", face_img)

detected = detect_face(face_img)
cv2.imshow("detected", detected)

cv2.waitKey(50)
cv2.destroyAllWindows()

/var/folders/6t/lw9p33tn1q124mgvz_z7v9gm0000gn/T/ipykernel_5528/3461351378.py:4: RuntimeWarning: divide by zero encountered in divide
  skin_mask = (S >= 50) & (0.5 < L / S) & (L / S < 3) & ((H <= 14) | (H >= 165))


##  Zadanie 6 (2 punkty)
Proszę dokonać detekcję efektu czerwonych oczu na zdjęciu "face1.jpg".

HINT - czerwone oczy znajdują się tam, gdzie jest spełniona następująca reguła:
$$
\left( L \geq 64 \right) \land \left(S \geq 100\right) \land \left(0.5 < \frac{L}{S} < 1.5 \right) \land \left( \left(H \leq 7\right) \lor \left(H \geq 162\right) \right) 
$$

![alt](https://raw.githubusercontent.com/przem85/PGiM/master/img/face1.jpg)

In [8]:
def detect_red_eyes(image):
    new_image = cv2.cvtColor(image, cv2.COLOR_BGR2HLS)
    # still getting divide by 0 warning?
    new_image_proper = np.maximum(new_image, 1)
    
    H, L, S = cv2.split(new_image)
    red_eye_mask = (L >= 64) & (S >= 100) & (0.5 < L/S) & (L/S < 1.5) & ((H <= 7) | (H >= 162))    
    
    # why no work? :(
    # new_image[~red_eye_mask] = [0, 0, 0]
    # return new_image
    
    red_eyes_detection = np.zeros_like(face_img)
    red_eyes_detection[red_eye_mask] = image[red_eye_mask]
    return red_eyes_detection
    
face_img = cv2.imread("./img/face.jpg", cv2.IMREAD_COLOR)
cv2.imshow("original", face_img)

detected = detect_red_eyes(face_img)
cv2.imshow("detected", detected)

cv2.waitKey(50)
cv2.destroyAllWindows()

/var/folders/6t/lw9p33tn1q124mgvz_z7v9gm0000gn/T/ipykernel_5528/1907108049.py:6: RuntimeWarning: divide by zero encountered in divide
  red_eye_mask = (L >= 64) & (S >= 100) & (0.5 < L/S) & (L/S < 1.5) & ((H <= 7) | (H >= 162))
